In [29]:
# This notebook integrates HYCOM sea-surface temperature (SST) data in Google Earth Engine (GEE) with animal telemetry data that is NOT in GEE.
# To get the track data into GEE, I converted it from csv to pandas DataFrame to geopandas GeoDataFrame to GEE FeatureClass.
# The integrated data is visualized as an animation using the time_slider.

# GEE's time_slider can animate a single layer. Therefore, the challenge is to combine the following into a single layer:
# a) The static track (i.e. all the points that comprise the track);
# b) The time-varying location along the track i.e. the animal's location per frame;
# c) The time-varing SST i.e. for each frame, show the snapshot of (daily) SST for the same day as (b)

# To combine these 3 together, we use the Image.blend() method--per frame. However, this only works on raster images.
# Items (a) and (b) above are vector-based, and (c) is raster. So, before I can blend the 3 items together (per frame), I have to convert
# the vector-based items (feature classes) to images. 

# Another challenge to overcome is the display. The HYCOM SST is a single band image.
# For single-band images, the "palette" is used to define how band values get translated into colors
# So, when I convert the vector-based items (feature classes) to images, I'll also need to reduce them to a single band AND 
# make sure the value of that single band shows up well in the display relative to the SST, given the palette. That's a lot!

#-Apply filterBounds to collection i.e. the roi--why isn't this working? Need to clip?


In [1]:
# Imports
import ee
import geemap
import geemap.colormaps as cm
import geedim
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime
import numpy as np

In [2]:
# Custom import from a different folder
import sys
sys.path.append('../project')
import project_functions

In [3]:
ee.Authenticate()

True

In [4]:
m1 = geemap.Map()

In [5]:
# ROI
roi = m1.user_roi
if roi is None:
    roi = ee.Geometry.BBox(-78.0, 30.0, -70.0, 40.0)
    #roi = ee.Geometry.BBox(-77.0, 30.0, -60.0, 45.0)
    #m.add_layer(roi)
    m1.center_object(roi)
    

In [6]:
# Get a single animal track as a pandas DataFrame

# Specify fullpath to input csv file containing multiple animal tracks
# i.e. the csv downloaded from movebank.org and load it into a pandas DataFrame
track_csv_fullpath = '../../project_data/track_data/Short_finned_pilot_whales_CRC_NW_Atlantic.csv'

# Let's look at just one of the tracks. From data exploration, I found GmTag142 to be 
# interesting i.e. it went through my region of interest
tag_id = 'GmTag142'  # e.g. 'GmTag142' or 'GmTag137'

# Call function to get the individual track, as pandas dataframe
myTrack = project_functions.get_movebank_track_from_csv(track_csv_fullpath, tag_id)

# Subset myTrack in time
myTrack_in_time_window = myTrack['timestamp_datetime']<datetime(2016,1,6)   # boolean
myTrack_subset = myTrack[myTrack_in_time_window]

# Reset index
myTrack_subset.reset_index()

    event-id                timestamp  location-long  location-lat  \
0  369777537  2014-06-11 17:49:22.000        -74.763        35.936   
1  369777538  2014-06-11 19:23:15.000        -74.810        35.891   
2  369777539  2014-06-11 20:26:13.000        -74.983        35.878   
3  369777540  2014-06-11 21:58:56.000        -74.829        35.920   
4  369777543  2014-06-12 00:03:31.000        -74.859        35.935   

  individual-local-identifier  timestamp_datetime  
0                    GmTag092 2014-06-11 17:49:22  
1                    GmTag092 2014-06-11 19:23:15  
2                    GmTag092 2014-06-11 20:26:13  
3                    GmTag092 2014-06-11 21:58:56  
4                    GmTag092 2014-06-12 00:03:31  


,index,event-id,timestamp,location-long,location-lat,individual-local-identifier,timestamp_datetime
0,4466,1033855502,2015-10-21 14:26:58.000,-74.77900,35.62500,GmTag142,2015-10-21 14:26:58
1,4467,1033855503,2015-10-21 15:23:06.000,-74.77200,35.62200,GmTag142,2015-10-21 15:23:06
2,4468,1033855505,2015-10-21 17:39:48.000,-74.75700,35.62900,GmTag142,2015-10-21 17:39:48
3,4469,1033855506,2015-10-21 21:20:51.000,-74.77600,35.62900,GmTag142,2015-10-21 21:20:51
4,4470,1033855508,2015-10-21 21:57:22.000,-74.77400,35.62500,GmTag142,2015-10-21 21:57:22
...,...,...,...,...,...,...,...
508,4974,1125787087,2016-01-05 10:52:11.000,-73.58870,36.10448,GmTag142,2016-01-05 10:52:11
509,4975,1125787088,2016-01-05 11:33:59.000,-73.65079,36.11283,GmTag142,2016-01-05 11:33:59
510,4976,1125787089,2016-01-05 14:09:08.000,-73.57721,36.22884,GmTag142,2016-01-05 14:09:08
511,4977,1125787092,2016-01-05 22:12:45.000,-73.23096,36.47164,GmTag142,2016-01-05 22:12:45


In [7]:
# Convert track from pandas DataFrame to geopandas GeoDataFrame
# First, specify geometry
geometry_myTrack = gpd.points_from_xy(myTrack_subset['location-long'],myTrack_subset['location-lat'])
gdf_myTrack = gpd.GeoDataFrame(myTrack_subset,geometry=geometry_myTrack,crs="EPSG:4326")

In [8]:
# Reset index
gdf_myTrack.reset_index()
gdf_myTrack

,event-id,timestamp,location-long,location-lat,individual-local-identifier,timestamp_datetime,geometry
4466,1033855502,2015-10-21 14:26:58.000,-74.77900,35.62500,GmTag142,2015-10-21 14:26:58,POINT (-74.779 35.625)
4467,1033855503,2015-10-21 15:23:06.000,-74.77200,35.62200,GmTag142,2015-10-21 15:23:06,POINT (-74.772 35.622)
4468,1033855505,2015-10-21 17:39:48.000,-74.75700,35.62900,GmTag142,2015-10-21 17:39:48,POINT (-74.757 35.629)
4469,1033855506,2015-10-21 21:20:51.000,-74.77600,35.62900,GmTag142,2015-10-21 21:20:51,POINT (-74.776 35.629)
4470,1033855508,2015-10-21 21:57:22.000,-74.77400,35.62500,GmTag142,2015-10-21 21:57:22,POINT (-74.774 35.625)
...,...,...,...,...,...,...,...
4974,1125787087,2016-01-05 10:52:11.000,-73.58870,36.10448,GmTag142,2016-01-05 10:52:11,POINT (-73.5887 36.10448)
4975,1125787088,2016-01-05 11:33:59.000,-73.65079,36.11283,GmTag142,2016-01-05 11:33:59,POINT (-73.65079 36.11283)
4976,1125787089,2016-01-05 14:09:08.000,-73.57721,36.22884,GmTag142,2016-01-05 14:09:08,POINT (-73.57721 36.22884)
4977,1125787092,2016-01-05 22:12:45.000,-73.23096,36.47164,GmTag142,2016-01-05 22:12:45,POINT (-73.23096 36.47164)


In [9]:
# Convert geopandas track to GEE feature class and add it as a layer to the map
fc_my_Track = geemap.gdf_to_ee(gdf_myTrack)

In [10]:
geometry_myTrack


<GeometryArray>
[<POINT (-74.779 35.625)>, <POINT (-74.772 35.622)>, <POINT (-74.757 35.629)>,
 <POINT (-74.776 35.629)>, <POINT (-74.774 35.625)>, <POINT (-74.729 35.646)>,
 <POINT (-74.739 35.668)>,  <POINT (-74.77 35.685)>, <POINT (-74.774 35.691)>,
 <POINT (-74.778 35.698)>,
 ...
 <POINT (-74.847 35.665)>, <POINT (-73.698 36.154)>, <POINT (-73.681 36.166)>,
 <POINT (-73.656 36.175)>,  <POINT (-73.613 36.12)>, <POINT (-73.589 36.104)>,
 <POINT (-73.651 36.113)>, <POINT (-73.577 36.229)>, <POINT (-73.231 36.472)>,
 <POINT (-73.216 36.482)>]
Length: 513, dtype: geometry

In [11]:
# Extract array and convert to numpy 
df_timestamp_datetime = myTrack_subset['timestamp_datetime']
timestamp_datetime_array = np.array(df_timestamp_datetime)

In [12]:
# Try converting the feature collection to an image, using reduceToImage
# It needs properties. Try just using the timestamp_datetime field?
# Doesn't like that: Property 'timestamp_datetime' not a numeric type
# Try location-long?
try_track_image = fc_my_Track.reduceToImage(properties=['event-id'],reducer=ee.Reducer.first()).multiply(-1)
try_track_image

In [63]:
# Alternatively, try converting the feature collection to an image, using the draw method of the FeatureCollection
try_track_image_draw = fc_my_Track.draw(color='ff0000',pointRadius=1, strokeWidth=1)
try_track_image_draw_1 = try_track_image_draw.select("vis-red").multiply(1000)
try_track_image_draw_1

In [64]:
# Use vis params bounds that work for HYCOM SST image
# Note that the HYCOM SST image is a single band. 
# For single-band images, the palette is used to define how band values get translated into colors
min_val = (4-20)*1000
max_val = (27-20)*1000
vis_params_sst = {
  "min": min_val,
  "max": max_val,
  "palette": ['000000', '005aff', '43c8c8', 'fff700', 'ff0000'],
}

In [65]:
# Check conversion of feature collection to image
m2 = geemap.Map()

m2.add_layer(try_track_image_draw_1, vis_params_sst, "try_track_image")
m2

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [31]:
# A function that returns a single HYCOM image, given a date
# Caller must fix scale and offset
# For now, fudge variable date range
def single_HYCOM_SST_image(this_timestamp_datetime,roi):
    # HYCOM SST collection
    collection = (ee.ImageCollection('HYCOM/sea_temp_salinity')
        .filter(ee.Filter.date("2019-03-06","2019-03-07"))
        .select('water_temp_0')
        .filter(ee.Filter.bounds(roi))          
    )
    # return first image from the collection, scaled and offset to degrees C
    first_image = collection.first()
    #return first_image.multiply(0.001).add(20.0)
    # For some reason, the caller has trouble with max<min when I include the scaling and offset here;
    # therefore, let the caller apply the scaling and offset in the vis_params
    return first_image

In [68]:
# Initialize
image_list = []
# Loop on samples along the track--just a few for now until I get it working
for frame_num in range(1,len(timestamp_datetime_array)-1,10):
#for frame_num in range(4):    
    # Get the associated HYCOM SST image for this sample time
    this_timestamp_datetime = timestamp_datetime_array[frame_num].item()
    this_image = single_HYCOM_SST_image(this_timestamp_datetime,roi)
    # Merge the SST image i.e. "blend" it with the static image of the track
    blended_image1 = this_image.blend(try_track_image_draw_1)
    # Also convert this frame's point (along the track) into an image
    # First, extract this feature from the feature class
    this_timestamp_datetime_str = this_timestamp_datetime.strftime("%Y-%m-%dT%H:%M:%S")
    this_feature = fc_my_Track.filter(ee.Filter.eq('timestamp_datetime',this_timestamp_datetime_str))
    # Next, convert this single-feature feature class into a single-band image
    this_feature_image_draw = this_feature.draw(color='ff0000',pointRadius=3, strokeWidth=3)
    this_feature_image_draw_1 = this_feature_image_draw.select("vis-red").multiply(-1000)
    # Finally, blend this image as well
    blended_image2 = blended_image1.blend(this_feature_image_draw_1)
    # Append this image to the image list
    image_list.append(blended_image2)    

In [69]:
# Create an image collection from the image_list
custom_collection = ee.ImageCollection.fromImages(image_list).filter(ee.Filter.bounds(roi)) 

# New map
m3 = geemap.Map()

# Add slider
m3.add_time_slider(custom_collection, vis_params_sst, time_interval=1)
m3

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

In [ ]:
# Track is on top of SST...

In [ ]:
# Test function before putting it in the loop--no longer needed
this_image = single_HYCOM_SST_image(this_timestamp_datetime)

min_val = 4
max_val = 27
vis_params_sst = {
  "min": min_val,
  "max": max_val,
  "palette": ['000000', '005aff', '43c8c8', 'fff700', 'ff0000'],
}

#m.add_layer(this_image, vis_params, "single_SST")
#m